In [2]:
import time
import datetime
import argparse
import random
import os, sys


import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from matplotlib.ticker import NullLocator

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import numpy as np

from tqdm import tqdm

import io
import PIL
import requests
import cv2
import shutil

#from google.colab.patches import cv2_imshow
import os.path
import time
import shutil
import glob
import warnings

def clean_folder(folder_name):
    try:
      shutil.rmtree(folder_name)
    except OSError as e:
      pass
    os.makedirs(folder_name, exist_ok=True)

In [3]:
yolo_path = "detLib/"
sys.path.append(yolo_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from detLib.models import Darknet
from detLib.utils.utils import *
from detLib.utils.datasets import *

def y_path(p):
    return os.path.join(yolo_path,p)

weights = y_path("weights/yolov3_ckpt_current_50.pth")
model_def = y_path("config/yolov3-custom.cfg")
model = Darknet(model_def, img_size=416).to(device)
model.load_state_dict(torch.load(weights, map_location=device))
model.eval()
classes = ['pedestrian']
conf_thres, nms_thres = 0.9, 0.4

In [12]:
# une img

output_dir = "output/"
clean_folder(output_dir)

resizeSize = 960  # if zero keep original dimnesion
img_path = "Frames/MOT16-10-raw/frame1.jpg"

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = transforms.ToTensor()(img)
img, _ = pad_to_square(img, 0)

if resizeSize != 0:
    img = resize(img, resizeSize)
img.unsqueeze_(0)  # add batch axis

with torch.no_grad():
    start_time = time.time()
    detections = model(img)
    detections = non_max_suppression(detections, conf_thres,nms_thres)
    detections = detections[0]
    print(f"--- %s seconds for {img.shape}---" % (time.time() - start_time))

    start_time = time.time()
    #img = np.array(Image.open(img_path))
    img = cv2.imread(img_path)
    detections = rescale_boxes(detections, resizeSize, img.shape[:2])
    for i, (x1, y1, x2, y2, conf, cls_conf, cls_pred) in enumerate(detections):
        box_w = x2 - x1
        box_h = y2 - y1

        if 0.45*box_h > box_w > 10:
            pedestrian = img[int(y1):int(y2), int(x1):int(x2), :]
            filename =f'{output_dir}/{i:04}.png'
            cv2.imwrite(filename, pedestrian)
    print(f"--- %s seconds for saving {i} images---" % (time.time() - start_time))

--- 7.050244092941284 seconds for torch.Size([1, 3, 960, 960])---
--- 0.024682998657226562 seconds for saving 27 images---


In [15]:

videoname = 'MOT16-10-raw'
videoframe = cv2.VideoCapture('vids/'+videoname+'.webm')
output_dir = "output/"
clean_folder(output_dir)
resizeSize = 960  # if zero keep original dimnesion


framenr=0

if not videoframe.isOpened() :
  print("Error opening video stream or file")
while videoframe.isOpened():
  # Capture frame-by-frame
  ret, img = videoframe.read()
  if ret:
    framenr+=1

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = transforms.ToTensor()(img)
    img, _ = pad_to_square(img, 0)

    if resizeSize != 0:
        img = resize(img, resizeSize)
    img.unsqueeze_(0)  # add batch axis

    with torch.no_grad():
        start_time = time.time()
        detections = model(img)
        detections = non_max_suppression(detections, conf_thres,nms_thres)
        detections = detections[0]
        print(f"--- %s seconds for {img.shape}---" % (time.time() - start_time))

        start_time = time.time()
        #img = np.array(Image.open(img_path))
        img = cv2.imread(img_path)
        detections = rescale_boxes(detections, resizeSize, img.shape[:2])
        for i, (x1, y1, x2, y2, conf, cls_conf, cls_pred) in enumerate(detections):
            box_w = x2 - x1
            box_h = y2 - y1

            if 0.45*box_h > box_w > 10:
                pedestrian = img[int(y1):int(y2), int(x1):int(x2), :]
                filename =f'{output_dir}/{framenr}-{i:04}.png'
                cv2.imwrite(filename, pedestrian)
        print(f"--- %s seconds for saving {i} images---" % (time.time() - start_time))

        if framenr > 3:
            break

videoframe.release()
# Closes all the frames
cv2.destroyAllWindows()


--- 6.855797052383423 seconds for torch.Size([1, 3, 960, 960])---
--- 0.02375197410583496 seconds for saving 27 images---
--- 7.303478002548218 seconds for torch.Size([1, 3, 960, 960])---
--- 0.027131080627441406 seconds for saving 18 images---
--- 6.22790789604187 seconds for torch.Size([1, 3, 960, 960])---
--- 0.024360179901123047 seconds for saving 27 images---
--- 7.276079893112183 seconds for torch.Size([1, 3, 960, 960])---
--- 0.024101972579956055 seconds for saving 31 images---


In [ ]:
"""" some test to make the model work with full size image without padding, complicate to modify cause model is for square
from torchvision.transforms.functional import pad

s = list(file.size)
to_pad = tuple([int((31-(si-1)%32)/2) for si in s])
print(to_pad)

file = pad(file, padding=to_pad)
print(file.size)
"""
